### 静态图(TensorFlow 1.x)

In [8]:
import numpy as np
import torch

In [9]:
class OP:
    def __init__(self):
        self.name = self.__class__.__name__

    def __call__(self, *args):
        self.input = args
        self.output = Placeholder()
        self.output.op = self
        return self.output

    def compute(self):

        new_input = []
        for index, item in enumerate(self.input):  # input在推理的时候记录下来
            if isinstance(item, Tensor):
                if item.op is not None:
                    item = item.op.compute()  # 将placeholder转变为确定的值
            new_input.append(item)

        self.input = new_input
        output = self.forward(*new_input)
        output.op = self
        return output

    def forward(self, *args):
        raise NotImplementedError()

    def backward(self, *args):
        raise NotImplementedError()

    def backward_native(self, grad):
        # 可以把backward理解为求导
        input_grads = self.backward(grad)
        if not isinstance(input_grads, tuple):
            input_grads = (input_grads, )

        assert len(input_grads) == len(self.input), "Number grads mismatch number input"

        for ig, ip in zip(input_grads, self.input):
            if isinstance(ip, Tensor):
                ip.backward(ig)

    def get_data(self, item):
        if isinstance(item, Tensor):
            return item.data
        else:
            return item


class AddOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) + self.get_data(b))

    def backward(self, grad):
        '''
            分别返回对(a+b)对a和b求导的值
            d(a+b)/da = 1
            d(a+b)/db = 1
            所以(1*grad, 1*grad)
        '''
        return grad, grad


class SubOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) - self.get_data(b))

    def backward(self, grad):
        return grad, -1 * grad


class MulOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) * self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad * self.get_data(b), grad * self.get_data(a)


class DivOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) / self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad / self.get_data(b), grad * self.get_data(a) / (self.get_data(b) ** 2) * -1


class ExpOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.exp(self.get_data(a)))

    def backward(self, grad):
        return grad * np.exp(self.get_data(self.input[0]))


class LogOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.log(self.get_data(a)))

    def backward(self, grad):
        return grad / self.get_data(self.input[0])


class MatMulOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) @ self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad @ self.get_data(b).T, self.get_data(a).T @ grad


class SumOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.sum(self.get_data(a)))

    def backward(self, grad):
        a = self.input[0]
        return np.full_like(self.get_data(a), grad)


class MeanOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.mean(self.get_data(a)))

    def backward(self, grad):
        a = self.input[0]
        d = self.get_data(a)
        return np.full_like(d, grad / d.size)


class Tensor:
    def __init__(self, data, op=None):
        self.data = data
        self.op = op
        self.grad = 0

    def __radd__(self, other):
        return AddOP()(other, self)

    def __add__(self, other):
        return AddOP()(self, other)

    def __rsub__(self, other):
        return SubOP()(other, self)

    def __sub__(self, other):
        return SubOP()(self, other)

    def __rmul__(self, other):
        return MulOP()(other, self)

    def __mul__(self, other):
        return MulOP()(self, other)

    def __rtruediv__(self, other):
        return DivOP()(other, self)

    def __truediv__(self, other):
        return DivOP()(self, other)

    def __neg__(self):
        return MulOP()(self, -1)

    def __matmul__(self, other):
        return MatMulOP()(self, other)

    def __repr__(self):
        if self.op is not None:
            return f"tensor({self.data}, grad_fn=<{self.op.name}>)"
        else:
            return f"{self.data}"

    def backward(self, grad=1):
        self.grad = self.grad + grad  # 就是1
        if self.op is not None:
            self.op.backward_native(grad)


class Placeholder(Tensor):
    def __init__(self):
        super().__init__(0)


def SessionRun(var, feed_dict):
    for key in feed_dict:
        key.data = feed_dict[key]

    return var.op.compute()


# 模拟包的形式
class morch:

    @staticmethod
    def exp(value):
        return ExpOP()(value)

    @staticmethod
    def log(value):
        return LogOP()(value)

    @staticmethod
    def sum(value):
        return SumOP()(value)

    @staticmethod
    def mean(value):
        return MeanOP()(value)

In [2]:
import numpy as np
import torch

class OP:
    def __init__(self):
        self.name = self.__class__.__name__

    def __call__(self, *args):
        '''
            每一次eval表达式，不但把结果输出
            还把表达式本身放在结果里

            这可能是静态图的作用，因为变量的值是从顶层传入的（最后时刻传入）

            所以一个复杂表达式后:
            a = sum(1/e^(-x) @ M)
            先后__call__了这几个
                __neg__, __exp__, __truediv__, __matmul__, __sum__
            此时a.op就是__sum__这个operator, input(即参数)就是前面一层的__matmul__的output
            同理，__matmul__的input是__truediv__，
            这个遍历是在compute函数里进行的，所以只要在顶层调用compute，将会用input（即args）来
            依次往下遍历完。

            output.op是往上传的，所以通过每个入参的op能找到它的下一级
        '''
        self.input = args
        self.output = Placeholder()
        self.output.op = self
        return self.output

    def compute(self):

        new_input = []
        for index, item in enumerate(self.input):  # input在推理的时候记录下来
            if isinstance(item, Tensor):
                # item.op等于成了是否是计算结果还是单纯一个表达式的标识
                # 如果是放结果，op必然指向计算它的表达式
                if item.op is not None:
                    # 这里递归调到最底层的-a（此时a已经有值）, 依次把output层层计算上来
                    item = item.op.compute()  # 将placeholder转变为确定的值
            new_input.append(item)

        self.input = new_input
        output = self.forward(*new_input)
        output.op = self
        return output

    def forward(self, *args):
        raise NotImplementedError()

    def backward(self, *args):
        raise NotImplementedError()

    def backward_native(self, grad):
        # 可以把backward理解为求导
        input_grads = self.backward(grad)
        # 在例子中，标量结果来自于矩阵求和，这个方法就处于矩阵求和的方法体内
        # 找到矩阵求和的求导方法，就把导数包成了全1矩阵
        if not isinstance(input_grads, tuple):
            input_grads = (input_grads, )

        assert len(input_grads) == len(self.input), "Number grads mismatch number input"

        # 依然用这个例子，走到这一步，显然，只有一个入参，即矩阵
        # 再把这个全1的新梯度（即导数）往这个矩阵的底层回传
        # 因为矩阵来自于两个矩阵相乘 (a(1/e^-a) @ b)，
        # 可见，必然找到了__matmul__的函数体中去了 （错！）

        # 要知道只有Tensor才会调backward_native方法，而在这个方法里才会往参数里传入梯度
        # 以回传，所以如果这里就跑到operator里面去了，那肯定是不能BP下去了

        # 所以要知道，sum的矩阵，这个矩阵来自两个矩阵相乘没错
        # 但首先，它是一个矩阵，即，一个Tensor
        # 所以下面的代码在sum矩阵这一段其实是这个意思：
        # 入参是一个矩阵，检查这个矩阵是否是一个Tensor对象
        # 如果是，就进入该对象的backward方法
        # 順理成章，它就进入了该tensor产生来源(op)的native方法中
        # 这才回到这里，已经成了两个入参（矩阵）的一个乘法表达式了
        # 这时会返回两个梯度，分别是对左取导，对右取导的
        # 再判断左右参是否Tensor，继续回传

        # 所以整个BP是通过Tensor对象回传的，OP只是一个简单的操作符，
        # 但是却在这个操作符里通过参数，把梯度传下去
        for ig, ip in zip(input_grads, self.input):
            if isinstance(ip, Tensor):
                ip.backward(ig)

    def get_data(self, item):
        if isinstance(item, Tensor):
            return item.data
        else:
            return item


class AddOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) + self.get_data(b))

    def backward(self, grad):
        '''
            分别返回对(a+b)对a和b求导的值
            d(a+b)/da = 1
            d(a+b)/db = 1
            所以(1*grad, 1*grad)
        '''
        return grad, grad


class SubOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) - self.get_data(b))

    def backward(self, grad):
        return grad, -1 * grad


class MulOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) * self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad * self.get_data(b), grad * self.get_data(a)


class DivOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) / self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad / self.get_data(b), grad * self.get_data(a) / (self.get_data(b) ** 2) * -1


class ExpOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.exp(self.get_data(a)))

    def backward(self, grad):
        return grad * np.exp(self.get_data(self.input[0]))


class LogOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.log(self.get_data(a)))

    def backward(self, grad):
        return grad / self.get_data(self.input[0])


class MatMulOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a, b):
        return Tensor(self.get_data(a) @ self.get_data(b))

    def backward(self, grad):
        a, b = self.input
        return grad @ self.get_data(b).T, self.get_data(a).T @ grad


class SumOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.sum(self.get_data(a)))

    def backward(self, grad):
        '''
        在例子中，最顶层是对matrix求sum，得到一个标量:12.5306...
        所以对t = (....) 整个表达式求backward，就是对标量求导，当然为1
        但是这个标量是由sum得到的（从op可以取出），于是就调了op的backward_native

        backward_native做的第一件事就是先执行一遍自己的backward，
        所以就进了这个sum的backward的方法

        注意，只有Tensor对象才会去调这个native方法，op对象只会把计算出自身的表达式
        的参数，分别传入当然梯度，往下传播
        '''
        a = self.input[0]
        '''
        对矩阵的每一项求导，那么那一项就是1，其它项就成了常数，变成了0
        sum起来，就成了每一项都是1
        '''
        return np.full_like(self.get_data(a), grad)


class MeanOP(OP):
    def __init__(self):
        super().__init__()

    def forward(self, a):
        return Tensor(np.mean(self.get_data(a)))

    def backward(self, grad):
        a = self.input[0]
        d = self.get_data(a)
        return np.full_like(d, grad / d.size)


class Tensor:
    def __init__(self, data, op=None):
        self.data = data
        self.op = op
        self.grad = 0

    def __radd__(self, other):
        return AddOP()(other, self)

    def __add__(self, other):
        return AddOP()(self, other)

    def __rsub__(self, other):
        return SubOP()(other, self)

    def __sub__(self, other):
        return SubOP()(self, other)

    def __rmul__(self, other):
        return MulOP()(other, self)

    def __mul__(self, other):
        return MulOP()(self, other)

    def __rtruediv__(self, other):
        return DivOP()(other, self)

    def __truediv__(self, other):
        return DivOP()(self, other)

    def __neg__(self):
        return MulOP()(self, -1)

    def __matmul__(self, other):
        return MatMulOP()(self, other)

    def __repr__(self):
        if self.op is not None:
            return f"tensor({self.data}, grad_fn=<{self.op.name}>)"
        else:
            return f"{self.data}"

    def backward(self, grad=1):
        self.grad = self.grad + grad  # 就是1
        # 假如是一个表达式的计算结果，那么进入这个表达式的backward_native函数中去
        if self.op is not None:
            self.op.backward_native(grad)


class Placeholder(Tensor):
    def __init__(self):
        super().__init__(0)


def SessionRun(var, feed_dict):
    for key in feed_dict:
        key.data = feed_dict[key]

    return var.op.compute()


# 模拟包的形式
class morch:

    @staticmethod
    def exp(value):
        return ExpOP()(value)

    @staticmethod
    def log(value):
        return LogOP()(value)

    @staticmethod
    def sum(value):
        return SumOP()(value)

    @staticmethod
    def mean(value):
        return MeanOP()(value)


print("==========================================基于PyTorch的自动微分-动态图版本：==========================================")
a = torch.tensor(value, dtype=torch.float32, requires_grad=True)
b = torch.tensor(mul_value, dtype=torch.float32, requires_grad=True)
t = torch.sum(1 / (1 + torch.exp(-a)) @ b)
t.backward()

print("计算结果是：", t)
print("a的导数是：", a.grad.numpy())
print("b的导数是：", b.grad.numpy())

print("\n\n")


print("==========================================自己写的自动微分-静态图版本：==========================================")
# 当你执行完表达式时，就等同于构建了一个计算图。通过计算图反推即可得到梯度
a = Placeholder()   #  shape     None, H, W,  C
b = Tensor(mul_value)   #  可训练的参数，比如初始化为 高斯初始化，凯明初始化，常量初始化
t = morch.sum(1 / (1 + morch.exp(-a)) @ b)

# 构建计算图

out = SessionRun(t, {a: value})
out.backward()

print("计算结果是：", out)
print("dt/da：", a.grad)
print("dt/db：", b.grad)

==========================================基于PyTorch的自动微分-动态图版本：==========================================
计算结果是： tensor(12.53067, grad_fn=<SumBackward0>)
a的导数是： [[0.09375 0.29492 0.27561]
 [0.01694 0.02649 0.01745]
 [0.00092 0.00137 0.00088]]
b的导数是： [[2.4501  2.4501  2.4501 ]
 [2.71216 2.71216 2.71216]
 [2.87377 2.87377 2.87377]]



==========================================自己写的自动微分-静态图版本：==========================================
计算结果是： tensor(12.530673399567604, grad_fn=<SumOP>)
dt/da： [[0.09375 0.29492 0.27561]
 [0.01694 0.02649 0.01745]
 [0.00092 0.00137 0.00088]]
dt/db： [[2.4501  2.4501  2.4501 ]
 [2.71216 2.71216 2.71216]
 [2.87377 2.87377 2.87377]]
